<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук ВШЭ, программист-исследователь Mail.ru Group  Юрий Кашницкий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Занятие 5. Композиции алгоритмов, случайный лес</center>
## <center>Практика. Деревья решений и случайный лес в соревновании Kaggle Inclass по кредитному скорингу. Решение</center>

**[Соревнование](https://inclass.kaggle.com/c/beeline-credit-scoring-competition-2).**

Решается задача кредитного скоринга. 

Признаки клиентов банка:
- Age - возраст (вещественный)
- Income - месячный доход (вещественный)
- BalanceToCreditLimit - отношение баланса на кредитной карте к лимиту по кредиту (вещественный)
- DIR - Debt-to-income Ratio (вещественный)
- NumLoans - число заемов и кредитных линий
- NumRealEstateLoans - число ипотек и заемов, связанных с недвижимостью (натуральное число)
- NumDependents - число членов семьи, которых содержит клиент, исключая самого клиента (натуральное число)
- Num30-59Delinquencies - число просрочек выплат по кредиту от 30 до 59 дней (натуральное число)
- Num60-89Delinquencies - число просрочек выплат по кредиту от 60 до 89 дней (натуральное число)
- Delinquent90 - были ли просрочки выплат по кредиту более 90 дней (бинарный) - имеется только в обучающей выборке

In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Загружаем данные.**

In [4]:
train_df = pd.read_csv('../../data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('../../data/credit_scoring_test.csv', index_col='client_id')

In [5]:
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [6]:
train_df.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350


**Посмотрим на число пропусков в каждом признаке.**

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 0 to 74999
Data columns (total 9 columns):
DIR                      75000 non-null float64
Age                      75000 non-null float64
NumLoans                 75000 non-null int64
NumRealEstateLoans       75000 non-null int64
NumDependents            73084 non-null float64
Num30-59Delinquencies    75000 non-null int64
Num60-89Delinquencies    75000 non-null int64
Income                   60153 non-null float64
BalanceToCreditLimit     75000 non-null float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 75000 to 149999
Data columns (total 9 columns):
DIR                      75000 non-null float64
Age                      75000 non-null float64
NumLoans                 75000 non-null int64
NumRealEstateLoans       75000 non-null int64
NumDependents            72992 non-null float64
Num30-59Delinquencies    75000 non-null int64
Num60-89Delinquencies    75000 non-null int64
Income                   60116 non-null float64
BalanceToCreditLimit     75000 non-null float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


**Заменим пропуски медианными значениями.**

In [9]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)

### Дерево решений без настройки параметров

**Обучите дерево решений максимальной глубины 3, используйте параметр random_state=42 для воспроизводимости результатов.**

In [8]:
first_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
first_tree.fit(train_df, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best')

**Сделайте прогноз для тестовой выборки.**

In [9]:
first_tree_pred = first_tree.predict(test_df)

**Запишем прогноз в файл.**

In [10]:
def write_to_submission_file(predicted_labels, out_file,
                             target='Delinquent90', index_label="client_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(75000, 
                                                  predicted_labels.shape[0] + 75000),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [11]:
write_to_submission_file(first_tree_pred, '../../output/credit_scoring_first_tree.csv')

**На публичной части тестовой выборки на Kaggle результат у этой посылки - 0.538 ROC AUC.**

**Если предсказывать вероятности дефолта для клиентов тестовой выборки, результат будет намного лучше.**

In [12]:
first_tree_pred_probs = first_tree.predict_proba(test_df)[:, 1]

In [13]:
write_to_submission_file(first_tree_pred_probs, '../../output/credit_scoring_first_tree_prob.csv')

**На публичной части тестовой выборки на Kaggle результат у этой посылки ~ 0.81 ROC AUC.**

## Дерево решений с настройкой параметров с помощью GridSearch

**Настройте параметры дерева с помощью GridSearh, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр random_state=42 (для воспроизводимости результатов), не забывайте про распараллеливание (n_jobs=-1).**

In [12]:
tree_params = {'max_depth': list(range(3,8)), 
               'min_samples_leaf': list(range(5,13))}

locally_best_tree = GridSearchCV(DecisionTreeClassifier(random_state=42), 
                                 tree_params, 
                                 verbose=True, n_jobs=-1, cv=5,
                                scoring='roc_auc')
locally_best_tree.fit(train_df, y)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   25.5s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 4, 5, 6, 7], 'min_samples_leaf': [5, 6, 7, 8, 9, 10, 11, 12]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc',
       verbose=True)

In [13]:
locally_best_tree.best_params_, round(locally_best_tree.best_score_, 3)

({'max_depth': 6, 'min_samples_leaf': 12}, 0.829)

In [14]:
from sklearn.tree import export_graphviz
export_graphviz(locally_best_tree.best_estimator_, feature_names=train_df.columns, 
                out_file='../../output/cred_scoring_tree.dot')
!dot -Tpng '../../output/cred_scoring_tree.dot' -o '../../img/cred_scoring_tree.png'

<img src='../../img/cred_scoring_tree.png'>

**Сделайте прогноз для тестовой выборки и пошлите решение на Kaggle.**

In [15]:
tuned_tree_pred_probs = locally_best_tree.predict_proba(test_df)[:, 1]

In [17]:
write_to_submission_file(tuned_tree_pred_probs, '../../output/credit_scoring_tuned_tree.csv')

**На публичной части тестовой выборки на Kaggle результат у этой посылки - 0.841 ROC AUC.**

### Случайный лес без настройки параметров

**Обучите случайный лес из деревьев максимальной глубины 3, используйте параметр random_state=42 для воспроизводимости результатов.**

In [18]:
first_forest = RandomForestClassifier(max_depth=3, random_state=42)
first_forest.fit(train_df, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [19]:
first_forest_pred = first_forest.predict_proba(test_df)[:, 1]

**Сделайте прогноз для тестовой выборки и пошлите решение на Kaggle.**

In [20]:
write_to_submission_file(first_forest_pred, '../../output/credit_scoring_first_forest.csv')

**На публичной части тестовой выборки на Kaggle результат у этой посылки ~ 0.84 ROC AUC.**

### Случайный лес c настройкой параметров

**Настройте параметры леса с помощью GridSearh, посмотрите на лучшую комбинацию параметров и среднее качество на 5-кратной кросс-валидации. Используйте параметр random_state=42 (для воспроизводимости результатов), не забывайте про распараллеливание (n_jobs=-1).**

In [16]:
%%time
forest_params = {'max_depth': list(range(1,11)),
                 'min_samples_leaf': list(range(1,11))}

locally_best_forest = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), 
                                 forest_params, 
                                 verbose=True, n_jobs=-1, cv=5,
                                  scoring='roc_auc')
locally_best_forest.fit(train_df, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   39.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.8min finished


CPU times: user 15 s, sys: 3.32 s, total: 18.3 s
Wall time: 2min 50s


In [17]:
locally_best_forest.best_params_, round(locally_best_forest.best_score_, 3)

({'max_depth': 9, 'min_samples_leaf': 5}, 0.844)

In [18]:
tuned_forest_pred = locally_best_forest.predict_proba(test_df)[:, 1]

In [25]:
write_to_submission_file(tuned_forest_pred, '../../output/credit_scoring_tuned_forest.csv')

**На публичной части тестовой выборки на Kaggle результат у этой посылки ~ 0.85 ROC AUC.**

**Посмотрите, как настроенный случайный лес оценивает важность признаков по их влиянию на целевой. Представьте результаты в наглядном виде с помощью DataFrame.**

In [19]:
pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_,
             index=train_df.columns,
            columns=['Importance']).sort(['Importance'], ascending=False)

,Importance
BalanceToCreditLimit,0.252400
Num60-89Delinquencies,0.249520
Num30-59Delinquencies,0.214966
Age,0.075354
NumLoans,0.060568
DIR,0.059265
Income,0.058033
NumRealEstateLoans,0.016069
NumDependents,0.013823


**Обычно увеличение количества деревьев только улучшает результат. Так что напоследок обучите случайный лес из 300 деревьев с найденными лучшими параметрами. Это может занять несколько минут.**

In [ ]:
%%time
final_forest = RandomForestClassifier(n_estimators=300, max_depth=9, min_samples_leaf=5,
                                      random_state=42, n_jobs=-1)
final_forest.fit(train_df, y)
final_forest_pred = final_forest.predict_proba(test_df)[:, 1]
write_to_submission_file(final_forest_pred, '../../output/credit_scoring_final_forest.csv')

**Сделайте посылку на Kaggle.**

**На публичной части тестовой выборки на Kaggle результат у этой посылки ~ 0.857 ROC AUC.**